이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter11_part02_sequence-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

### 단어를 시퀀스로 처리하기: 시퀀스 모델 방식

#### 첫 번째 예제

**데이터 다운로드**

In [22]:
# !rm -r aclImdb
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup

**데이터 준비**

In [23]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    if not (val_dir / category).exists():
        os.makedirs(val_dir / category) 
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 6554 files belonging to 2 classes.
Found 18446 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**정수 시퀀스 데이터셋 준비하기**

In [24]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

2024-06-10 00:33:16.389132: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


**원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델 만들기**

In [39]:
# 커스텀 레이어 정의
class OneHotLayer(layers.Layer):
    def __init__(self, depth, **kwargs):
        super(OneHotLayer, self).__init__(**kwargs)
        self.depth = depth

    def call(self, inputs):
        return tf.one_hot(inputs, depth=self.depth)

    def get_config(self):
        config = super(OneHotLayer, self).get_config()
        config.update({"depth": self.depth})
        return config

inputs = keras.Input(shape=(max_length,), dtype="int64")
embedded = OneHotLayer(depth=max_tokens)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 600)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ one_hot_layer (OneHotLayer)     │ (None, 600, 20000)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 64)             │     5,128,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,128,513 (19.56 MB)

 Trainable params: 5,128,513 (19.56 MB)

 Non-trainable params: 0 (0.00 B)

**첫 번째 시퀀스 모델 훈련하기**

In [42]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 54s 261ms/step - accuracy: 0.5017 - loss: 0.6917 - val_accuracy: 0.7164 - val_loss: 0.6074
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 53s 259ms/step - accuracy: 0.7374 - loss: 0.5784 - val_accuracy: 0.8303 - val_loss: 0.4216
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 52s 257ms/step - accuracy: 0.8453 - loss: 0.4138 - val_accuracy: 0.8615 - val_loss: 0.3493
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 52s 255ms/step - accuracy: 0.8899 - loss: 0.3189 - val_accuracy: 0.8643 - val_loss: 0.3692
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 52s 256ms/step - accuracy: 0.9098 - loss: 0.2605 - val_accuracy: 0.8592 - val_loss: 0.4373
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 52s 256ms/step - accuracy: 0.9234 - loss: 0.2603 - val_accuracy: 0.8660 - val_loss: 0.3809
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 53s 258ms/step - accuracy: 0.9467 - loss: 0.1744 - val_accuracy: 0.8586 - val_loss: 0.3382
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 53s 257ms/step - accuracy: 0.9485 - loss: 0

In [44]:
# 모델 로드 시 커스텀 레이어를 등록
model = keras.models.load_model("one_hot_bidir_lstm.keras", custom_objects={"OneHotLayer": OneHotLayer})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 54ms/step - accuracy: 0.8521 - loss: 0.3460
테스트 정확도: 0.849


#### 단어 임베딩 이해하기

#### 임베딩 층으로 단어 임베딩 학습하기

**`Embedding` 층 만들기**

In [45]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**밑바닥부터 훈련하는 `Embedding` 층을 사용한 모델**

In [46]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 64)             │        73,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 34s 162ms/step - accuracy: 0.5468 - loss: 0.6802 - val_accuracy: 0.7030 - val_loss: 0.5757
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 161ms/step - accuracy: 0.7613 - loss: 0.5261 - val_accuracy: 0.7322 - val_loss: 0.5431
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 34s 164ms/step - accuracy: 0.8337 - loss: 0.4013 - val_accuracy: 0.7789 - val_loss: 0.4727
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 162ms/step - accuracy: 0.8894 - loss: 0.2887 - val_accuracy: 0.8124 - val_loss: 0.4632
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 161ms/step - accuracy: 0.8982 - loss: 0.2689 - val_accuracy: 0.8190 - val_loss: 0.5056
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 162ms/step - accuracy: 0.9252 - loss: 0.2048 - val_accuracy: 0.8313 - val_loss: 0.4728
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 161ms/step - accuracy: 0.9511 - loss: 0.1485 - val_accuracy: 0.7451 - val_loss: 0.6455
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 160ms/step - accuracy: 0.9566 - loss: 0

#### 패딩과 마스킹 이해하기

**마스킹을 활성화한 `Embedding` 층 사용하기**

In [47]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_with_masking.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_17      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 256) │  5,120,000 │ input_layer_17[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer_17[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_10    │ (None, 64)        │     73,984 │ embedding_4[0][0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ bidirectional_10… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │         65 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 36s 173ms/step - accuracy: 0.5768 - loss: 0.6584 - val_accuracy: 0.8092 - val_loss: 0.4475
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 170ms/step - accuracy: 0.8212 - loss: 0.4115 - val_accuracy: 0.7840 - val_loss: 0.4579
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8734 - loss: 0.3015 - val_accuracy: 0.8528 - val_loss: 0.3618
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 36s 176ms/step - accuracy: 0.9117 - loss: 0.2225 - val_accuracy: 0.8379 - val_loss: 0.4254
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 171ms/step - accuracy: 0.9306 - loss: 0.1846 - val_accuracy: 0.8560 - val_loss: 0.4000
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 172ms/step - accuracy: 0.9481 - loss: 0.1393 - val_accuracy: 0.8500 - val_loss: 0.4106
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 171ms/step - accuracy: 0.9667 - loss: 0.0893 - val_accuracy: 0.8590 - val_loss: 0.4763
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 169ms/step - accuracy: 0.9703 - loss: 0

**첫 번째 시퀀스 모델 훈련하기**

#### 사전 훈련된 단어 임베딩 사용하기

In [48]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

--2024-06-10 01:06:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-06-10 01:06:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-10 01:06:48--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**GloVe 단어 임베딩 파일 파싱하기**

In [49]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"단어 벡터 개수: {len(embeddings_index)}")

단어 벡터 개수: 400000


**GloVe 단어 임베딩 행렬 준비하기**

In [50]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [51]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**사전 훈련된 임베딩을 사용하는 모델**

In [52]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, None, 100) │  2,000,000 │ input_layer_18[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ input_layer_18[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_11    │ (None, 64)        │     34,048 │ embedding_5[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 64)        │          0 │ bidirectional_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         65 │ dropout_9[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,034,113 (7.76 MB)

 Trainable params: 34,113 (133.25 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 38s 182ms/step - accuracy: 0.5370 - loss: 0.7017 - val_accuracy: 0.6599 - val_loss: 0.6040
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 37s 181ms/step - accuracy: 0.7067 - loss: 0.5794 - val_accuracy: 0.7013 - val_loss: 0.5561
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 37s 180ms/step - accuracy: 0.7514 - loss: 0.5213 - val_accuracy: 0.7451 - val_loss: 0.4968
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 169ms/step - accuracy: 0.7734 - loss: 0.4729 - val_accuracy: 0.6899 - val_loss: 0.6472
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 37s 180ms/step - accuracy: 0.7934 - loss: 0.4519 - val_accuracy: 0.7973 - val_loss: 0.4420
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 169ms/step - accuracy: 0.8168 - loss: 0.4113 - val_accuracy: 0.8128 - val_loss: 0.4458
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 37s 182ms/step - accuracy: 0.8204 - loss: 0.3999 - val_accuracy: 0.8259 - val_loss: 0.3932
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 35s 170ms/step - accuracy: 0.8345 - loss: 0